In [1]:
using DataFrames, Query
using CSV

In [9]:
df = CSV.File("data/titanic.csv") |> DataFrame;

In [11]:
first(df, 10)

,PassengerId,Survived,Pclass,Name
,Int64,String,Int64,String
1,1,FALSE,3,"Braund, Mr. Owen Harris"
2,2,TRUE,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"
3,3,TRUE,3,"Heikkinen, Miss. Laina"
4,4,TRUE,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
5,5,FALSE,3,"Allen, Mr. William Henry"
6,6,FALSE,3,"Moran, Mr. James"
7,7,FALSE,1,"McCarthy, Mr. Timothy J"
8,8,FALSE,3,"Palsson, Master. Gosta Leonard"
9,9,TRUE,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"


In [26]:
@time males = df |> @filter(_.Sex == "male") |> DataFrame;
nrow(males)

  0.065509 seconds (133.60 k allocations: 7.127 MiB, 18.13% gc time)


577

In [25]:
%time

The analogue of IPython's `%time statement` (also `%timeit`) in Julia is `@time statement`.  The analogue of `%%time ...code...` is

```
@time begin
    ...code...
end
```

Note, however, that you should put all performance-critical code into a function, avoiding global variables, before doing performance measurements in Julia; see the [performance tips in the Julia manual](http://docs.julialang.org/en/latest/manual/performance-tips/).

The `@time` macro prints the timing results, and returns the value of evaluating the expression.  To instead return the time (in seconds), use `@elapsed statement`.

For more extensive benchmarking tools, including the ability to collect statistics from multiple runs, see the [BenchmarkTools package](https://github.com/JuliaCI/BenchmarkTools.jl).


In [40]:
using Statistics

In [90]:
sexes = df |> @groupby("class "*string(_.Pclass)*" "*_.Sex) |> @map({Key=key(_), Count=length(_), Age=mean(_.Age)}) |> DataFrame

,Key,Count,Age
,String,Int64,Float64
1,class 1 female,74,34.0946
2,class 1 male,84,40.6717
3,class 3 female,5,17.2
4,class 2 male,6,18.75
5,class 2 female,9,29.6111
6,class 3 male,5,24.8


In [69]:
mean(df.Age)

35.6744262295082

In [68]:
dropmissing!(df)

,PassengerId,Survived,Pclass,Name
,Int64,String,Int64,String
1,2,TRUE,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"
2,4,TRUE,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
3,7,FALSE,1,"McCarthy, Mr. Timothy J"
4,11,TRUE,3,"Sandstrom, Miss. Marguerite Rut"
5,12,TRUE,1,"Bonnell, Miss. Elizabeth"
6,22,TRUE,2,"Beesley, Mr. Lawrence"
7,24,TRUE,1,"Sloper, Mr. William Thompson"
8,28,FALSE,1,"Fortune, Mr. Charles Alexander"
9,53,TRUE,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)"


In [58]:
df |> @filter(_.Survived == "TRUE") |> DataFrame |> nrow

342

# IP Addrs

In [ ]:
ipcsv = CSV.File("/home/zach/Downloads/traffic_2", header=["IP", "domain"])

In [3]:
ipdf = DataFrame!(ipcsv, ["IP", "domain"])

UndefVarError: UndefVarError: ipcsv not defined

In [ ]:
ipdf |> @groupby()

# JuliaDB IP Addrs

In [1]:
using Distributed

In [3]:
if nprocs() == 1
    addprocs(11);
end

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [5]:
using JuliaDB

In [18]:
using OnlineStats, ProgressMeter

In [8]:
fn = "/home/zach/Downloads/loads/traffic_2"

"/home/zach/Downloads/traffic_2"

In [34]:
o = HyperLogLog(SubString{String})
p = ProgressUnknown("Items Processed:")
for line in eachline(fn)
    IP,domain = split(line, '\t')
    fit!(o, IP)
    ProgressMeter.next!(p)
    if o.n > 100_000_000
        break
    end
end

Items Processed: 99905977 	 Time: 0:01:01

In [ ]:
ips = loadtable("/home/zach/Downloads/traffic_2"; output="/scratch/bin", chunks=1000)

In [ ]:
ips = load("/scratch/bin")